In [1]:
pip install wikipedia-api


Note: you may need to restart the kernel to use updated packages.


In [2]:
import wikipediaapi
from transformers import pipeline
import pandas as pd


2022-04-22 13:11:08.600418: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 13:11:08.600538: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
wiki_wiki = wikipediaapi.Wikipedia('fr')
page_py = wiki_wiki.page('David Beckham')
df_test_DB=pd.DataFrame(page_py.text.split(','))
df_test_DB

,0
0,David Robert Joseph Beckham
1,né le 2 mai 1975 à Leytonstone (Londres)
2,est un footballeur international anglais ayan...
3,au Real Madrid
4,à l'AC Milan
...,...
461,2003. (ISBN 0-06-057093-8)\nMichael Crick
462,The Boss – The Many Sides of Alex Ferguson
463,Pocket Books
464,2003. (ISBN 0-7434-2991-5)\n\nLiens internes...


In [4]:
df_test_DB.to_csv('DB_test.csv')

In [20]:
pipeline = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
# You can then use this pipeline to classify sequences into any of the class names you specify.
candidate_labels = ['personal data', 'not personal data']

In [15]:
# def preprocess(self, dataframe):


#     self.log.info("In preprocess function.")

#     dataframe1 = self.remove_nan(dataframe)
#     dataframe2 = self.lowercase(dataframe1)
#     dataframe3 = self.remove_whitespace(dataframe2)

#     # Remove emails and websites before removing special characters
#     dataframe4 = self.remove_emails(self, dataframe3)
#     dataframe5 = self.remove_website_links(self, dataframe4)

#     dataframe6 = self.remove_special_characters(dataframe5)
#     dataframe7 - self.remove_numbers(dataframe6)
#     self.remove_stop_words(dataframe8) # Doesn't return anything for now
#     dataframe7 = self.tokenize(dataframe6)

#     self.log.info(f"Sample of preprocessed data: {dataframe4.head()}")

#     return dataframe7

# def remove_nan(self, dataframe):
#     """Pass in a dataframe to remove NAN from those columns."""
#     return dataframe.dropna()

# def lowercase(self, dataframe):
#     logging.info("Converting dataframe to lowercase")
#     lowercase_dataframe = dataframe.apply(lambda x: x.lower())
#     return lowercase_dataframe


# def remove_special_characters(self, dataframe):
#     self.log.info("Removing special characters from dataframe")
#     no_special_characters = dataframe.replace(r'[^A-Za-z0-9 ]+', '', regex=True)
#     return no_special_characters

# def remove_numbers(self, dataframe):
#     self.log.info("Removing numbers from dataframe")
#     removed_numbers = dataframe.str.replace(r'\d+','')
#     return removed_numbers

# def remove_whitespace(self, dataframe):
#     self.log.info("Removing whitespace from dataframe")
#     # replace more than 1 space with 1 space
#     merged_spaces = dataframe.str.replace(r"\s\s+",' ')
#     # delete beginning and trailing spaces
#     trimmed_spaces = merged_spaces.apply(lambda x: x.str.strip())
#     return trimmed_spaces

# def remove_stop_words(self, dataframe):
#     # TODO: An option to pass in a custom list of stopwords would be cool.
#     set(stopwords.words('english'))

# def remove_website_links(self, dataframe):
#     self.log.info("Removing website links from dataframe")
#     no_website_links = dataframe.str.replace(r"http\S+", "")
#     return no_website_links

# def tokenize(self, dataframe):
#     tokenized_dataframe = dataframe.apply(lambda row: word_tokenize(row))
#     return tokenized_dataframe

# def remove_emails(self, dataframe):
#     no_emails = dataframe.str.replace(r"\S*@\S*\s?")
#     return no_emails

# def expand_contractions(self, dataframe):
#     # TODO: Not a priority right now. Come back to it later.
#     return dataframe

In [28]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    # sentence = sentence.lower()
    # sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    # rem_url=re.sub(r'http\S+', '',cleantext)
    # rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    # tokens = tokenizer.tokenize(rem_num)  
    # filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    # stem_words=[stemmer.stem(w) for w in filtered_words]
    # lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return cleantext




In [29]:
df_test_DB['cleanText']=df_test_DB[0].map(lambda s:preprocess(s)) 

In [27]:
# labels=df_test_DB['cleanText'].apply(lambda x: pipeline(x, candidate_labels))
# labels

In [30]:
for x in df_test_DB['cleanText']:
    try:
        print(pipeline(x, candidate_labels))
    except:
        print(x)

{'sequence': 'David Robert Joseph Beckham', 'labels': ['personal data', 'not personal data'], 'scores': [0.736955463886261, 0.263044536113739]}
{'sequence': ' né le 2 mai 1975 à Leytonstone (Londres)', 'labels': ['personal data', 'not personal data'], 'scores': [0.7468413710594177, 0.25315865874290466]}
{'sequence': ' est un footballeur international anglais ayant évolué au poste de milieu de terrain à Manchester United', 'labels': ['personal data', 'not personal data'], 'scores': [0.678486168384552, 0.3215138912200928]}
{'sequence': ' au Real Madrid', 'labels': ['not personal data', 'personal data'], 'scores': [0.5424549579620361, 0.45754507184028625]}
{'sequence': " à l'AC Milan", 'labels': ['not personal data', 'personal data'], 'scores': [0.5446047782897949, 0.45539525151252747]}
{'sequence': " au Galaxy de Los Angeles et au Paris Saint-Germain.\nSa carrière débute lorsqu'il signe à Manchester United et fait sa première apparition dans l'équipe première en 1992 à l'âge de dix-sept 